### Transfer Learning

-  zzimcong dataset에 맞게 Inception V3 모델을 이용한 전이학습

In [17]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

import pandas as pd

In [18]:
path = f'C:/Users/sanghui/Desktop/toyproject/DL_Project_5/hsh/data/crop_resize/train/2'

for category_id in os.listdir(path):
    print(category_id, len(os.listdir(f'{path}/{category_id}')))


11 1774
12 3056
13 3035
14 3048
15 2828
16 2934
17 2974
18 3020
19 789
25 3140
39 3134
40 1566
46 646


In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input

In [23]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=[0.5, 0.7], # 이미지 줌
    brightness_range=[0.2, 0.5], # 이미지 밝기
    preprocessing_function=preprocess_input
)

In [24]:
train_generator = train_datagen.flow_from_directory('./data/crop_resize/train/2', target_size=(299, 299), batch_size=60, class_mode='categorical')

Found 31944 images belonging to 13 classes.


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from tensorflow.keras import applications

In [26]:
input_shape = (299, 299, 3)
nclass = len(train_generator.class_indices)

base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dropout(0.5))
add_model.add(Dense(nclass, activation='softmax'))

model = add_model

model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])
model.summary()



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d_5   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_3 (Dropout)         (None, 2048)              0         
                                                                 
 dense_3 (Dense)             (None, 13)                26637     
                                                                 
Total params: 21,829,421
Trainable params: 26,637
Non-trainable params: 21,802,784
_________________________________________________________________


C:\Users\sanghui\miniconda3\envs\ds_study\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [27]:
hist = model.fit_generator(train_generator, epochs=10, shuffle=True, verbose=True)

C:\Users\sanghui\AppData\Local\Temp/ipykernel_8104/2178757453.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(train_generator, epochs=10, shuffle=True, verbose=True)


Epoch 1/10
 82/533 [===>..........................] - ETA: 20:13 - loss: 2.6249 - accuracy: 0.0886

KeyboardInterrupt: 